# Compare the Afifi and Lakhnawi editions of the Fusus

In [1]:
%load_ext autoreload
%autoreload 2

In [119]:
from Levenshtein import distance, ratio

In [3]:
from tf.app import use

In [4]:
BASE = "~/github/among/fusus"
VERSION = "0.7"

# Load both editions

Normally, when we load a single data source in a notebook, we store the handle in a variable called
`A`, and we hoist additional variables `F`, `L`, `T`, etc to the global namespace.

But now we work with two datasources, so we store the handles in a dictionary `A`, with
a key `L` for the Lakhnawi edition and a key `A` for the Afifi edition.

We also make dictionaries for `F`, `L`, `T`, etc, keyed with the same keys.

In that way we can systematically select our handles for the desired editions.

In [5]:
LK = "LK"
AF = "AF"

EDITIONS = {
    LK: "Lakhnawi",
    AF: "Afifi",
}

A = {}
F = {}
E = {}
L = {}
T = {}
N = {}

In [6]:
for (acro, name) in EDITIONS.items():
    A[acro] = use(f"among/fusus/tf/{name}:clone", writing="ara", version=VERSION)
    F[acro] = A[acro].api.F
    E[acro] = A[acro].api.E
    L[acro] = A[acro].api.L
    T[acro] = A[acro].api.T
    N[acro] = A[acro].api.N

This is Text-Fabric 9.1.3
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

27 features found and 0 ignored


This is Text-Fabric 9.1.3
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

17 features found and 0 ignored


Let's find out the max slot of both editions.

In [7]:
maxSlot = {acro: F[acro].otype.maxSlot for acro in EDITIONS}
maxSlot

{'LK': 40379, 'AF': 40271}

We set up our comparison.

We work with the latin transcriptions, in order to avoid complications with right-to-left writing in 
the displays of situations where discrepancies occur.

The result of the comparison will be a mapping from Afifi slots to Lakhnawi slots.

Some slots are mapped to multiple slots, and some slots will not be mapped.
So it is convenient to also generate the inverse of the mapping.

In [8]:
getTextLK = F[LK].lettersn.v
getTextAF = F[AF].lettersn.v

maxLK = maxSlot[LK]
maxAF = maxSlot[AF]

mapping = {}
mappingInv = {}

comparison = []
indexLK = {}
indexAF = {}

We define auxiliary functions for finding discrepancies and inspecting them.

In [78]:
def printLines(start=0, end=None):
    if start < 0:
        start = 0
    if end is None or end > len(comparison):
        end = len(comparison)
    lines = []
    for (iLK, left, distance, right, iAF) in comparison[start:end]:
        textLK = getTextLK(iLK) if iLK else ""
        textAF = getTextAF(iAF) if iAF else ""
        lines.append(f"{iLK:>5} {left:<2} {textLK:>20} {distance:>2} {textAF:<20} {right:>2} {iAF:>5}")
    return "\n".join(lines)
        
        
def printComparison(path):
    with open(path, "w") as fh:
        fh.write(printLines())
        fh.write("\n")

            
def printDiff(before, after):
    print(printLines(start=len(comparison) - before))
    lastLK = None
    lastAF = None
    for c in range(len(comparison) - 1, -1, -1):
        comp = comparison[c]
        if lastLK is None:
            if comp[0]:
                lastLK = comp[0]
        if lastAF is None:
            if comp[4]:
                lastAF = comp[4]
        if lastLK is not None and lastAF is not None:
            break
    if lastLK is not None and lastAF is not None:
        for i in range(after):
            iLK = lastLK + 1 + i
            iAF = lastAF + 1 + i
            textLK = getTextLK(iLK) if iLK <= maxLK else ""
            textAF = getTextAF(iAF) if iAF <= maxAF else ""
            print(f"{iLK:>5} =  {textLK:>20} ?? {textAF:<20}  = {iAF:>5}")

Now the proper algorithm.

We stop when we cannot solve a discrepancy.

When solving discrepancies, we adjust the mapping and we record the severity of the
discrepancy in a separate dict `dissimilarity`.

We need to compute whether $n$ consecutive words left are similar to $m$ consecutive words
right.

We assume there is a boundary of *C* words that we will combine.

We need to walk to al possible combinations, from simplest and shortest to longest and most complex.

Every combination can be characterized by $(n, m)$, where $n$ is the number of words on the left
and $m$ is the number of words on the right. $n$ and $m$ are in the range $1 \ldots C$.

Suppose $C = 3$, then we want to compare combinations in the following order:

combination|x
---|---
$(1, 1)$|--
$(1, 2)$|--
$(2, 1)$|--
$(2, 2)$|--
$(1, 3)$|--
$(3, 1)$|--
$(2, 3)$|--
$(3, 2)$|--
$(3, 3)$|--

In fact, we list all possible combinations and then sort them first by sum of the pair 
and then by decreasing difference of the pair.

We fix a `C` (called `COMBI`) and compute the sequence of combinations up front.

In [79]:
def getCombis(c):
    combis = []
    for i in range(1, c + 1):
        for j in range(1, c + 1):
            if i != 1 or j != 1:
                combis.append((i, j))
    return tuple(sorted(combis, key=lambda x: (x[0] + x[1], abs(x[0] - x[1]))))

In [80]:
COMBINE = 4

COMBIS = getCombis(COMBINE)
COMBIS

((1, 2),
 (2, 1),
 (2, 2),
 (1, 3),
 (3, 1),
 (2, 3),
 (3, 2),
 (1, 4),
 (4, 1),
 (3, 3),
 (2, 4),
 (4, 2),
 (3, 4),
 (4, 3),
 (4, 4))

In [132]:
def similar(s1, s2, strictness):
    if s1 == s2:
        return (True, 0)
    
    d = distance(s1, s2)
    if type(strictness) is int:
        return (d <= strictness, d)
    else:
        return (ratio(s1, s2) >= strictness, d)

def findCombi(iLK, iAF, strictness):
    found = None
    
    for (cLK, cAF) in COMBIS:
        if iLK + cLK > maxLK or iAF + cAF > maxLK:
            continue
        textLK = "".join(getTextLK(iLK + i) for i in range(cLK))
        textAF = "".join(getTextAF(iAF + i) for i in range(cAF))
        (isSimilar, d) = similar(textLK, textAF, strictness)
        if isSimilar:
            found = (cLK, cAF)
            common = min((cLK, cAF))
            for i in range(max((cLK, cAF))):
                nComparison = len(comparison)
                if i < common:
                    mapping[iLK + i] = [iAF + i]
                    comparison.append((iLK + i, f"+{cLK}", f"@{d}", f"{cAF}+", iAF + i))
                    indexLK[iLK + i] = nComparison
                    indexAF[iAF + i] = nComparison
                elif i < cLK:
                    mapping[iLK + i] = [iAF + cAF]
                    comparison.append((iLK + i, f"+{cLK}", f"@{d}", f"{cAF}^", ""))
                    indexLK[iLK + i] = nComparison
                else:
                    mapping.setdefault(iLK + cLK, []).append(iAF + i)
                    comparison.append(("", f"^{cLK}", f"@{d}", f"{cAF}+", iAF + i))
                    indexAF[iAF + i] = nComparison
            break
    return found
        
    
def doCase(iLK, iAF):
    if iLK not in cases:
        return None
    
    (cLK, cAF) = cases[iLK]
    common = min((cLK, cAF))
    for i in range(max((cLK, cAF))):
        nComparison = len(comparison)
        if i < common:
            mapping[iLK + i] = [iAF + i]
            comparison.append((iLK + i, f"+{cLK}", "~~", f"{cAF}+", iAF + i))
            indexLK[iLK + i] = nComparison
            indexAF[iAF + i] = nComparison
        elif i < cLK:
            mapping[iLK + i] = [iAF + cAF]
            comparison.append((iLK + i, f"+{cLK}", "~~", f"{cAF}^", ""))
            indexLK[iLK + i] = nComparison
        else:
            mapping.setdefault(iLK + cLK, []).append(iAF + i)
            comparison.append(("", f"^{cLK}", "~~", f"{cAF}+", iAF + i))
            indexAF[iAF + i] = nComparison
    return (iLK + cLK, iAF + cAF)
    
    
def compare(iLK, iAF, strictness):
    """Strictness is edit distance if it is an integer, otherwise it is ratio
    """
    textLK = getTextLK(iLK)
    textAF = getTextAF(iAF)
    (isSimilar, d) = similar(textLK, textAF, strictness)
    if isSimilar:
        mapping[iLK] = [iAF]
        nComparison = len(comparison)
        comparison.append((iLK, "=", f"@{d}", "=", iAF))
        indexLK[iLK] = nComparison
        indexAF[iAF] = nComparison
        return (iLK + 1, iAF + 1)

    combi = findCombi(iLK, iAF, strictness)
    if combi is not None:
        (cLK, cAF) = combi
        return (iLK + cLK, iAF + cAF)
    
    return None
            
    
def lookup(iLK, iAF, strictness, start, end):
    step = None
    
    for i in range(start, end + 1):
        prevComparisonIndex = len(comparison)
        
        if iAF + i <= maxAF:
            step = compare(iLK, iAF + i, strictness)
            if step:
                thisComparison = list(comparison[prevComparisonIndex:])
                comparison[prevComparisonIndex:] = []
                
                for j in range(iAF, i):
                    indexAF[j] = len(comparison)
                    comparison.append(("", "-", "@99", "=", j))
                for thisComp in thisComparison:
                    nComparison = len(comparison)
                    thisLK = thisComp[0]
                    thisAF = thisComp[4]
                    if thisLK:
                        indexLK[thisLK] = nComparison
                    if thisAF:
                        indexAF[thisAF] = nComparison
                    comparison.append(thisComp)
                break

        if iLK + i <= maxLK:
            step = compare(iLK + i, iAF, strictness)
            if step:
                thisComparison = list(comparison[prevComparisonIndex:])
                comparison[prevComparisonIndex:] = []
                
                for j in range(iLK, i):
                    indexLK[j] = len(comparison)
                    comparison.append((j, "=", "@99", "-", ""))
                for thisComp in thisComparison:
                    nComparison = len(comparison)
                    thisLK = thisComp[0]
                    thisAF = thisComp[4]
                    if thisLK:
                        indexLK[thisLK] = nComparison
                    if thisAF:
                        indexAF[thisAF] = nComparison
                    comparison.append(thisComp)
                break
    return step
                
        
def doDiffs():
    mapping.clear()
    comparison.clear()
    
    iLK = 1
    iAF = 1

    complete = False
    
    while True:
        if iLK > maxLK or iAF > maxAF:
            complete = True
            break
            
        # if iLK > 60:
        #    break
            
        strictness = 1
        
        step = doCase(iLK, iAF)
        if step:
            (iLK, iAF) = step
            continue
            
        step = compare(iLK, iAF, strictness)
        if step:
            (iLK, iAF) = step
            continue
            
        step = lookup(iLK, iAF, strictness, 1, 10)
        if step:
            (iLK, iAF) = step
            continue
            
        strictness = 2
        
        step = compare(iLK, iAF, strictness)
        if step:
            (iLK, iAF) = step
            continue
            
        step = lookup(iLK, iAF, strictness, 1, 10)
        if step:
            (iLK, iAF) = step
            continue
            
        strictness = 3
        
        step = compare(iLK, iAF, strictness)
        if step:
            (iLK, iAF) = step
            continue
            
        step = lookup(iLK, iAF, strictness, 1, 5)
        if step:
            (iLK, iAF) = step
            continue
            
        strictness = 0.5
        
        step = compare(iLK, iAF, strictness)
        if step:
            (iLK, iAF) = step
            continue
            
        step = lookup(iLK, iAF, strictness, 1, 5)
        if step:
            (iLK, iAF) = step
            continue
            
        strictness = 1
        
        step = lookup(iLK, iAF, strictness, 10, 1000)
        if step:
            (iLK, iAF) = step
            continue
            
        edit
            
        break
            
    mappingInv.clear()
    for (i, js) in mapping.items():
        for j in js:
            mappingInv.setdefault(j, []).append(i)
            
    if complete:
        printComparison("zipLK-AF-complete.txt")
        print("Mapping complete")
    else:
        print("BLOCKED:")
        printDiff(20, 20)
        printComparison("zipLK-AF-incomplete.txt")

# Run the comparison

Here we go!

In [133]:
cases = {
    13539: (2, 1),
}

In [134]:
doDiffs()

Mapping complete


In [129]:
len(mapping)

39824

In [130]:
len(mappingInv)

39308

In [131]:
print(printLines(start=13310, end=13330))

13372 =                 mʿḳwl  0 mʿḳwl                 = 13361
13373 =                 wālḥḳ  0 wālḥḳ                 = 13362
13374 =                 mḥsws  0 mḥsws                 = 13363
13375 =                 mšhwd  0 mšhwd                 = 13364
13376 =                   ʿnd  0 ʿnd                   = 13365
13377 =              ālmʾmnyn  0 ālmʾmnyn              = 13366
13378 =                  wāhl  0 wāhl                  = 13367
13379 =                 ālkšf  0 ālkšf                 = 13368
13380 =               wālwǧwd  0 wālwǧwd               = 13369
13381 =                   wmā  0 wmā                   = 13370
13382 =                   ʿdā  0 ʿdā                   = 13371
13383 =                  hḏyn  0 hḏyn                  = 13372
13384 =               ālṣnfyn  0 ālṣnfyn               = 13373
13385 =                 fālḥḳ  0 fālḥḳ                 = 13374
13386 =                 ʿndhm  0 ʿndhm                 = 13375
13387 =                 mʿḳwl  0 mʿḳwl                 

The mapping itself is needed elsewhere in Text-Fabric, let us write it to file.
We write it as an edge feature into the AF edition.

In [74]:
edge = {}

In [75]:
def edgeFromMap():
    edge.clear()
    print("Make edge from slot mapping")

    for iLK in range(1, maxSlot[LK]+ 1):
        iAF = mapping[iLK]
        k = dissimilarity.get(iLK, None)
        if k is None:
            if iLK in edge:
                if iAF not in edge[iLK]:
                    edge[iLK][iAF] = None
            else:
                edge.setdefault(iLK, {})[iAF] = None
        else:
            if k > 0:
                for j in range(iAF, iAF + k + 1):
                    edge.setdefault(iLK, {})[j] = k
            elif k < 0:
                for i in range(iLK, iLK - k + 1):
                    edge.setdefault(i, {})[iA] = k
            else:
                edge.setdefault(iLK, {})[iAF] = 0

# Earlier attempts

In [9]:
def inspect(start, end):
    """Helper function for inspecting the situation in a given range of slots.
    
    Parameters
    ----------
    start: integer
        Slot number where we start the inspection.
    end: integer
        Slot number where we end the inspection.
        
    Returns
    -------
    None
        The situation will be printed as a table with a row for each slot
        and columns:
        slot number in LK,
        letters of that slot in LK
        letters of the corresponding slot in AF
    """
    for iLK in range(start, end):
        iAF = mapping[iLK]
        print(
            "{:>6}: {:<8} {:<8}".format(
                iLK,
                F[LK].lettersn.v(iLK),
                F[AF].lettersn.v(iAF),
            )
        )


def firstDiff(start):
    """Find the first discrepancy after a given position.
    
    First we walk quickly through the slots of LK, until we reach the starting position.
    
    Then we continue walking until the current slot is either
    
    *   a special case
    *   a discrepancy
    
    Parameters
    ----------
    start: integer
        start position
    
    Returns
    -------
    int or None
        If there is no discrepancy, None is returned,
        otherwise the position of the first discrepancy.
    """

    fDiff = None
    for iLK in range(1, maxSlot[LK] + 1):
        if iLK < start:
            continue
        iAF = mapping[iLK]
        lettersLK = F[LK].lettersn.v(iLK)
        lettersAF = F[AF].lettersn.v(iAF)
        if iLK in casesLK or iAF in casesAF or lettersLK != lettersAF:
            fDiff = iLK
            break
    return fDiff


def printDiff(slotLK, k):
    """Prints the situation around a discrepancy.
    
    Parameters
    ----------
    slotLK: integer
        position of the discrepancy
    k: integer
        amount of slots around the discrepancy to include in the display
        
    Returns
    -------
    A plain text display of the situation around the discrepancy.
    """
    
    comps = {}
    
    # gather the comparison material in comps
    # which will be a list of display items
    
    slotAF = mapping[slotLK]
    
    for iLK in range(max((1, slotLK - k)), min((maxSlot[LK], slotLK + k)) + 1):
        iAF = mapping.get(iLK, None)
        currentLK = iLK == slotLK
        currentAF = iAF == slotAF

        lettersLK = F[LK].lettersn.v(iLK)
        lettersAF = F[AF].lettersn.v(iAF)

        comps.setdefault(LK, []).append((lettersLK, currentLK))
        comps.setdefault(AF, []).append((lettersAF, currentAF))
        
    # turn the display items into strings and store them in rep
    # which is also keyed by the versions
    
    rep = {}
    for acro in comps:
        rep[acro] = printEdition(acro, comps[acro])

    # compose the display out of the strings per edition
    # and make a header of sectional information and slot positions
    
    print(
        """{} {}:{} ==> slotLK {} ==> {}
    {}
    {}
""".format(
            *T[acro].sectionFromNode(slotLK),
            slotLK,
            slotAF,
            rep[LK],
            rep[AF],
        )
    )


def printEdition(acro, comps):
    """Generate a string displaying a stretch of slots around a position.
    
    Parameters
    ----------
    comps: list of tuple
        For each slot there is a comp tuple consisting of
        
        *   the letters of the slot
        *   whether the slot is in the discrepancy position
        
    Returns
    -------
    string
        A sequence of words with boundary characters in between.
    """
    
    rep = ""
    for (letters, isCurrent) in comps:
        if letters is None:
            letters = "?"
        rep += f"▶{letters}◀" if isCurrent else f"╋{letters}"
    rep += "╋"
    return rep

In [112]:
MAX_ITER = 100

dissimilarity = {}

def doDiffs():
    global mapping
    
    mapping = {iLK: iLK for iLK in range(1, maxSlot[LK] + 1)}
    
    dissimilarity.clear()

    iteration = 0
    start = 1

    solved = True
    
    lastApplied = (None, None, None)

    while True:
        # try to find the next difference from where you are now
        iLK = firstDiff(start)

        if iLK is None:
            print(f"No more differences.\nFound {iteration} discrepancies")
            break

        if iteration > MAX_ITER:
            print("There might be more discrepancies: increase MAX_ITER")
            break

        iteration += 1
        
        # there is a discrepancy: we have to do work
        # we print it as a kind of logging
        
        printDiff(iLK, 8)

        # we try to solve the discrepancy
        # first we gather the information of about the slots at this position in both versions
    
        iAF = mapping[iLK]
        
        lettersLK = F[LK].lettersn.v(iLK)
        lettersAF = F[AF].lettersn.v(iAF)
        
        # and at the next position
        
        lettersNextLK = F[LK].lettersn.v(iLK + 1)
        lettersNextAF = F[AF].lettersn.v(iAF + 1)
        
        # the discrepancy is not solved unless we find it in a case or in a rule
        solved = None
        side = None
        skip = 0
        
        # first check the explicit cases the the LK side
        
        if iLK in casesLK:
            (action, param) = casesLK[iLK]
            (lastILK, lastAction, lastSide) = lastApplied
            
            if action == "skipother" and not (iLK == lastILK and action == lastAction and side == LK):
                plural = "" if param == 1 else "s"
                solved = f"{action} {param} slot{plural}"
                side = LK
                for m in range(iLK, maxSlot[LK] + 1):
                    mapping[m] += param
            elif action == "skipme":
                plural = "" if param == 1 else "s"
                solved = f"{action} {param} slot{plural}"
                side = LK
                skip = param - 1
                for m in range(maxSlot[LK], iLK + param - 1, -1):
                    mapping[m] = mapping[m - param]
                for m in range(iLK, iLK + param):
                    mapping[m] = None
            elif action == "collapse":
                plural = "" if param == 1 else "s"
                solved = f"{action} {param} fewer slot{plural}"
                side = LK
                dissimilarity[iLK] = -param
                skip = param
                for m in range(maxSlot[LK], iLK + param, -1):
                    mapping[m] = mapping[m - param]
                for m in range(iLK + 1, iLK + param + 1):
                    mapping[m] = mapping[iLK]
            elif action == "split":
                plural = "" if param == 1 else "s"
                solved = f"{action} into {param} extra slot{plural}"
                side = LK
                dissimilarity[iLK] = param
                for m in range(iLK + 1, maxSlot[LK] + 1):
                    mapping[m] += param
            elif action == "ok":
                solved = "incidental variation in word"
                side = LK
                dissimilarity[iLK] = 0
        elif lettersLK in casesLK:
            (action, param) = casesLK[lettersLK]
            if action == "ok":
                if lettersAF == param:
                    solved = "systematic variation in lexeme"
                    side = LK
                    dissimilarity[iLK] = 0
            elif action == "split":
                plural = "" if param == 1 else "s"
                solved = f"systematic {action} into {param} extra slot{plural}"
                side = LK
                dissimilarity[iLK] = param
                for m in range(iLK + 1, maxSlot[LK] + 1):
                    mapping[m] += param
                    
        # then check the explicit cases the the AF side
        
        elif lettersAF in casesAF:
            (action, param) = casesAF[lettersAF]
            if action == "skipme" and not (iLK == lastILK and action == lastAction and side == AF):
                plural = "" if param == 1 else "s"
                solved = f"{action} {param} slot{plural}"
                side = AF
                for m in range(iLK, maxSlot[LK] + 1):
                    mapping[m] += param
                    
        # now apply some generic rules
        
        elif lettersLK.replace("y", "w") == lettersAF.replace("y", "w"):
            solved = f"y/w equivalent"
            side = None
            dissimilarity[iLK] = 0
            
        # automatic split
        
        elif lettersLK == lettersAF + lettersNextAF:
            action = "split"
            param = 1
            solved = f"automatic {action} into {param} extra slot{plural}"
            side = None
            dissimilarity[iLK] = param
            for m in range(iLK + 1, maxSlot[LK] + 1):
                mapping[m] += param
                
        # automatic collapse
        
        elif lettersLK + lettersNextLK == lettersAF:
            action = "collapse"
            param = 1
            solved = f"automatic {action} {param} fewer slot{plural}"
            side = LK
            dissimilarity[iLK] = -param
            skip = param
            for m in range(maxSlot[LK], iLK + param, -1):
                mapping[m] = mapping[m - param]
            for m in range(iLK + 1, iLK + param + 1):
                mapping[m] = mapping[iLK]
            
        # automatic skipother
        
        elif lettersLK == lettersNextAF and not (iLK == lastILK and "skipother" == lastAction and side == LK):
            action = "skipother"
            param = 1
            plural = "" if param == 1 else "s"
            solved = f"automatic {action} {param} slot{plural}"
            side = LK
            for m in range(iLK, maxSlot[LK] + 1):
                mapping[m] += param
                
        # automatic skipme
        
        elif lettersNextLK == lettersAF:
            action = "skipme"
            param = 1
            plural = "" if param == 1 else "s"
            solved = f"{action} {param} slot{plural}"
            side = LK
            skip = param - 1
            for m in range(maxSlot[LK], iLK + param - 1, -1):
                mapping[m] = mapping[m - param]
            for m in range(iLK, iLK + param):
                mapping[m] = None
        else:
            setLK = set(lettersLK)
            setAF = set(lettersAF)
            if (len(setLK) >= 2 or len(setAF) >= 2) and len(setLK - setAF) <= 1 and len(setAF - setLK) <= 1:
                solved = f"single letter variation"
                side = None
                dissimilarity[iLK] = 1
            elif (len(setLK) >= 3 or len(setAF) >= 3) and len(setLK - setAF) <= 2 and len(setAF - setLK) <= 2:
                solved = f"double letter variation"
                side = None
                dissimilarity[iLK] = 2
                    
        if solved:
            lastApplied = (iLK, action, side)
            
        print(f"Action: {solved if solved else 'BLOCKED'}\n")

        # stop the loop if the discrepancy is not solved
        # The discrepancy has already been printed to the output,
        # so you can see immediately what is happening there
        
        if not solved:
            break

        # if the discrepancy was solved, 
        # advance to the first position after the discrepancy
        # and try to find a new discrepancy in the next iteration
        start = iLK + 1 + skip

    if not solved:
        print(f"{lettersLK=} {lettersNextLK=}")
        print(f"{lettersAF=} {lettersNextAF=}")
        print(f"Blocking difference in {iteration} iterations")

In [123]:
casesLK = {
    1: ("skipother", 2),
    115: ("skipother", 1),
    179: ("skipme", 1),
    204: ("skipme", 1),
    217: ("skipother", 1),
    248: ("skipme", 1),
    "ālmḥrm": ("ok", "mḥrm"),
    "ālḥkm": ("ok", "ālḥk"),
    "llh": ("ok", "lh"),
    "mmd": ("ok", "md"),
}

casesAF = {
    "ā": ("skipme", 1),
}

In [220]:
def printDiff(slotLK, slotAF, k, file=None):
    """Prints the situation around a discrepancy.
    
    You can specify the slot positions for both 
    editions or for just one of them.
    If you want to leave a slot position unspecified,
    pass None for it.
    
    Parameters
    ----------
    slotLK: integer
        position of the discrepancy in LK
    slotAF: integer
        position of the discrepancy in AF
    k: integer
        amount of slots around the discrepancy to include in the display
        If None, there is no limit.
    file: string, optional None
        If a string, the output is written to a file with that name,
        otherwise the file is printed to the standard output
        
    Returns
    -------
    A plain text display of the situation around the discrepancy.
    """
    
    if file is None:
        handle = sys.stdout.write
    else:
        fh = open(file, "w")
        handle = fh
        
    def log(msg):
        handle.write(f"{msg}\n")
        
    if slotLK is None and slotAF is None:
        slotLK = 1
        slotAF = 1
    elif slotLK is None:
        slotLK = findCorresponding(slotAF, mappingInv)
        if slotLK is None:
            return
    elif slotAF is None:
        slotAF = findCorresponding(slotLK, mapping)
        if slotAF is None:
            return
        
    if k is None:
        fromK = 1
        toK = maxLK
    else:
        fromK = max((1, slotLK - k))
        toK = min((maxLK, slotLK + k)) + 1
            
    useIAF = None
    doneAF = set()
    for iLK in range(fromK, toK):
        iAFs = mapping.get(iLK, [])
        isCurrentLK = "▶" if iLK == slotLK else "━"
        
        firstLeft = f"{iLK:>5} {isCurrentLK} {getTextLK(iLK):>20} ╋ "
        restLeft = f"{'●':>28} ╋ "
        
        iAF = iAFs[0] if len(iAFs) else None
        
        if iAF is None:
            if useIAF is None:
                textAF = "●"
            else:
                useIAF += 1
                textAF = f"●{getTextAF(useIAF)}"
        else:
            useIAF = iAF
            textAF = getTextAF(useIAF)
            
        isCurrentAF = "▶" if useIAF == slotAF else "━"
        firstRight = f"{textAF:<20} {isCurrentAF} {useIAF or '?':>5}"
        log(f"{firstLeft}{firstRight}")
        if len(iAFs) > 1:
            wasIAF = iAFs[0]
            for iAF in iAFs[1:]:
                for j in range(wasIAF + 1, iAF):
                    log(f"{restLeft}{getTextAF(j):<20} {isCurrentAF} {j:>5}")
                wasiAF = iAF
                log(f"{restLeft}{getTextAF(iAF):<20} {isCurrentAF} {iAF:>5}")
                
    if file:
        handle.close()
                
                
def findCorresponding(i, corr):
    j = None
    for k in range(1, 1000):
        js = corr.get(i + k, [])
        if len(js):
            j = js[0]
            break
        js = corr.get(i - k, [])
        if len(js):
            j = js[0]
            break
    if j is None:
        print(f"Cannot find corresponding item for {i}")
    return j